# Import Liblary

In [ ]:
!pip install transformers datasets evaluate rouge_score

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
!pip install accelerate>=0.20.1

In [ ]:
! pip install rouge

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import pandas as pd

import torch
import datasets
from datasets import Dataset
from datasets import load_metric

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load Liputan-6 Dataset

In [ ]:
canonical_train = pd.read_csv('dataset//canonical_train.csv')
canonical_test = pd.read_csv('dataset/canonical_test.csv')
canonical_dev = pd.read_csv('dataset/canonical_dev.csv')

## Split data

Pada project 2 text summartization, karena keterbatan device dan waktu, maka dataset yang digunakan sebanyak 50% untuk data train, validasi, dan testing.

In [ ]:
canonical_test = canonical_test.head(5486)

In [ ]:
canonical_test.shape

(5486, 5)

In [ ]:
from sklearn.model_selection import train_test_split

# Bagi data menjadi 50% untuk pelatihan dan 50% untuk pengujian
canonical_train_50, test_data = train_test_split(canonical_train, test_size=0.5, random_state=42)

In [ ]:
# Bagi data menjadi 50% untuk pelatihan dan 50% untuk pengujian
canonical_test_50, test_data1 = train_test_split(canonical_test, test_size=0.5, random_state=42)

In [ ]:
# Bagi data menjadi 50% untuk pelatihan dan 50% untuk pengujian
canonical_dev_50, test_data2 = train_test_split(canonical_dev, test_size=0.5, random_state=42)

In [ ]:
canonical_train_50.shape, canonical_test_50.shape, canonical_dev_50.shape

((96941, 5), (5486, 5), (5486, 5))

# Preprocess

In [ ]:
from transformers import BertConfig, GPT2Config, EncoderDecoderModel, BertModel, GPT2LMHeadModel

#load the pre_trained models
encoder_config = BertConfig.from_pretrained('cahya/bert-base-indonesian-522M')
decoder_config = GPT2Config.from_pretrained('cahya/gpt2-small-indonesian-522M')

#Make sure that the decoder confiq is set as a decoder
decoder_config.is_decoder = True
decoder_config.add_cross_attention = True

#Create the BERT encoder and GPT-2 decoer with the
encoder = BertModel.from_pretrained('cahya/bert-base-indonesian-1.5G', config=encoder_config)
decoder = GPT2LMHeadModel.from_pretrained('cahya/gpt2-small-indonesian-522M', config=decoder_config)

#Combine the encoder and decoder into as Seq2Seq model
model = EncoderDecoderModel(encoder=encoder, decoder=decoder)

#Set the decoder_start_token_id
model.config.decoder_start_token_id=3

#Set the other generation parameters
model.config.eos_token_id=1
model.config.pad_token_id=2
model.config.max_lenght=40
model.config.min_lenght=20
model.config.no_repeat_gram_size=3
model.config.early_stopping=True
model.config.lenght_penalty=2.0
model.config.num_beams=10
gradient_checkpointing=True


Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at cahya/gpt2-small-indonesian-522M and are newly initialized: ['transformer.h.6.crossattention.q_attn.weight', 'transformer.h.1.ln_cross_attn.bias', 'transformer.h.7.crossattention.c_proj.bias', 'transformer.h.9.ln_cross_attn.bias', 'transformer.h.9.crossattention.q_attn.bias', 'transformer.h.8.crossattention.q_attn.bias', 'transformer.h.9.crossattention.c_attn.bias', 'transformer.h.0.crossattention.c_attn.bias', 'transformer.h.7.ln_cross_attn.weight', 'transformer.h.7.ln_cross_attn.bias', 'transformer.h.2.crossattention.c_proj.bias', 'transformer.h.11.crossattention.q_attn.bias', 'transformer.h.3.crossattention.c_proj.bias', 'transformer.h.6.ln_cross_attn.bias', 'transformer.h.2.ln_cross_attn.bias', 'transformer.h.11.crossattention.c_proj.weight', 'transformer.h.5.crossattention.q_attn.weight', 'transformer.h.5.crossattention.c_attn.weight', 'transformer.h.4.ln_cross_attn.weight', 'transformer.h.3.crossatt

In [ ]:
from transformers import GPT2Tokenizer, BertTokenizer

bert_tokenizer = BertTokenizer.from_pretrained('cahya/bert-base-indonesian-1.5G')

gpt2_tokenizer = GPT2Tokenizer.from_pretrained('cahya/gpt2-small-indonesian-522M')
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

model.config.decoder_start_token_id = gpt2_tokenizer.bos_token_id
model.config.eos_token_id = gpt2_tokenizer.eos_token_id
model.config.pad_token_id = gpt2_tokenizer.pad_token_id

In [ ]:
prefix = "summarize:"

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["clean_article"]]
    model_inputs = bert_tokenizer(inputs, max_length=512, truncation=True)

    with gpt2_tokenizer.as_target_tokenizer():
        labels = gpt2_tokenizer(examples["clean_summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
train = Dataset.from_pandas(canonical_train_50)
valid = Dataset.from_pandas(canonical_dev_50)
test = Dataset.from_pandas(canonical_test_50)

In [ ]:
print(test)

Dataset({
    features: ['id', 'url', 'clean_article', 'clean_summary', 'extractive_summary', '__index_level_0__'],
    num_rows: 5486
})


In [ ]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_dev = valid.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/96941 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/5486 [00:00<?, ? examples/s]

Map:   0%|          | 0/5486 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=bert_tokenizer, model=model)

# Evaluate

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = bert_tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, bert_tokenizer.pad_token_id)

    # Convert labels from token IDs to text tokens
    decoded_labels = []
    for label_ids in labels:
        decoded_label = bert_tokenizer.convert_ids_to_tokens(label_ids)
        decoded_label = " ".join(decoded_label)
        decoded_labels.append(decoded_label)

    # Process decoded predictions and labels
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Compute ROUGE metrics
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    return {key: value * 100 for key, value in result.items()}


In [ ]:
import gc

gc.collect()

346

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device ", device)
model = model.to(device)

torch.cuda.empty_cache()

device  cuda


# Train

In [ ]:
from transformers import  Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

# batch_size = 16

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/liputan_6_model",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    logging_steps=10,
    save_steps=16,
    # num_train_epochs=4,
    warmup_steps=1,
    max_steps=16,
    save_total_limit=1,
    weight_decay=0.01,
    # fp16=True
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
    data_collator=data_collator,
    tokenizer=bert_tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:639: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
10,0.332000,0.702130,20.979159,11.412761,20.650125,20.652174


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=16, training_loss=0.35100774466991425, metrics={'train_runtime': 2188.9469, 'train_samples_per_second': 0.029, 'train_steps_per_second': 0.007, 'total_flos': 39138644459520.0, 'train_loss': 0.35100774466991425, 'epoch': 0.0})

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1701489350.6ce18943f435.397.0:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Upload 28 LFS files:   0%|          | 0/28 [00:00<?, ?it/s]

events.out.tfevents.1701489445.6ce18943f435.397.1:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

events.out.tfevents.1701489487.6ce18943f435.397.2:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

events.out.tfevents.1701516584.6ce18943f435.12064.9:   0%|          | 0.00/9.19k [00:00<?, ?B/s]

events.out.tfevents.1701516722.6ce18943f435.12064.10:   0%|          | 0.00/9.66k [00:00<?, ?B/s]

events.out.tfevents.1701516852.6ce18943f435.12064.11:   0%|          | 0.00/9.70k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

'https://huggingface.co/verydwis/liputan_6_model/tree/main/'

# Inference

In [ ]:
text = "summarize: Undang-Undang Pengurangan Inflasi menurunkan biaya obat resep, biaya perawatan kesehatan, dan biaya energi. Ini adalah tindakan paling agresif dalam mengatasi krisis iklim dalam sejarah Amerika, yang akan meningkatkan kesejahteraan pekerja Amerika dan menciptakan pekerjaan bersama yang berbayar baik di seluruh negara. Ini akan menurunkan defisit dan meminta mereka yang sangat kaya dan perusahaan untuk membayar bagian yang adil. Dan tidak ada yang penghasilannya kurang dari $400.000 per tahun yang akan membayar lebih banyak pajak."

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="verydwis/liputan_6_model")
summarizer(text)

Pytorch

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("verydwis/liputan_6_model")
inputs = tokenizer(text, return_tensors="pt").input_ids


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/733k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("verydwis/liputan_6_model")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)